In [11]:
from ieasyhydro_sdk.sdk import IEasyHydroSDK
from dotenv import load_dotenv

from ieasyhydro_sdk.filters import BasicDataValueFilters
import datetime as dt

import pandas as pd
import sys
import os

# Get the absolute path of the directory containing the current script
cwd = os.getcwd()

# Construct the path to the iEasyHydroForecast directory
forecast_dir = os.path.join(
    cwd, '..', 'iEasyHydroForecast')

# Add the forecast directory to the Python path
sys.path.append(forecast_dir)

# Import the modules from the forecast library
import tag_library as tl
import forecast_library as fl

env_file_path = "../config/.env_develop_kghm"
load_dotenv(env_file_path)
print("DEBUG: IEASYHYDRO_HOST: ", os.getenv("IEASYHYDRO_HOST"))

# Load sdk configuration from .env
ieh_sdk = IEasyHydroSDK()

predictor_dates = [dt.datetime(2024, 5, 13, 0, 0, 0), dt.datetime(2024, 5, 15, 12, 0, 0)]
predictor_dates = [dt.datetime(2024, 4, 13, 0, 0, 0), dt.datetime(2024, 4, 17, 12, 0, 0)]

# Define date filter
filters = BasicDataValueFilters(
    local_date_time__gte=predictor_dates[0],
    local_date_time__lt=predictor_dates[1]
)

sites = ["15102", "15212", "15215", "15216", "15149", "15214", "15171",
                 "15194", "15189", "15292", "15261", "15954", "15960",
                 "16059", "16096", "16100", "16936"]
sites = ["16139"]

for site in sites:
    #site = '15102'

    try:
        # Get data
        qdata = ieh_sdk.get_data_values_for_site(
            [site],
            'discharge_daily_average',
            filters=filters
        )
        qdata = pd.DataFrame(qdata['data_values'])
        #print("get_data_values_for_site:\n", qdata)
        #print(type(qdata))

        tdata = ieh_sdk.get_data_values_for_site(
            [site],
            'discharge_daily',
            filters=filters,
        )
        tdata = pd.DataFrame(tdata['data_values'])
        #print(tdata)

        # Get the first row from tdata in the wide format
        row = pd.DataFrame(tdata.iloc[-1]).transpose()
        #print(row)

        # add the row to qdata
        qdata = pd.concat([qdata, row])
        print(qdata)
        print(f"{site} : {round(sum(qdata['data_value']), 3)}")

    except:
        print(f"site {site} not in DB")
        continue


DEBUG: IEASYHYDRO_HOST:  http://localhost:8881
     data_value     local_date_time       utc_date_time
0     53.168939 2024-04-13 14:00:00 2024-04-13 08:00:00
1     71.117942 2024-04-14 14:00:00 2024-04-14 08:00:00
2  19733.517565 2024-04-15 14:00:00 2024-04-15 08:00:00
3     98.637114 2024-04-16 14:00:00 2024-04-16 08:00:00
8     98.637114 2024-04-17 10:00:00 2024-04-17 04:00:00
16139 : 20055.079
